<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>CET Template Notebook</b> <br>
Contact author(s): <i>Author Name</i> <br>
Last verified to run: <i>yyyy-mm-dd</i> <br>
LSST Science Piplines version: Weekly <i>yyyy_xx</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>beginner</i> <br>

_In this template, text in italics are examples or instructions that should be: (a) removed if it is not applicable to the notebook; or (b) replaced with text that is appropriate for the notebook. But bold or regular text should appear pretty much as-is in all CET notebooks. For more information, see the [CET's Guidelines for Tutorial Notebooks](https://confluence.lsstcorp.org/pages/viewpage.action?pageId=168857070)._

_While developing, use the following code cell to check that the code conforms to standards, but then delete the cell and "Kernel --> Restart Kernel and Clear All Outputs" before saving and committing._

In [1]:
#%load_ext pycodestyle_magic
#%flake8_on
#import logging
#logging.getLogger("flake8").setLevel(logging.FATAL)

_The six cells below are considered the extended header of the notebook. The first four will be used, verbatim, to create the table of notebook metadata in the README.md file for the repository._

**Description:** _Very brief description of notebook._

**Skills:** _Brief list of skills to match the README.md file for the repository._

**LSST Data Products:** _List the all of the types of LSST catalogs and images used._

**Packages:** _List the python packages used._ (_List the packages being taught first, e.g., afwDisplay for a notebook about displaying images. Then supporting packages, e.g., lsst.daf.butler for a notebook about displaying images. It is OK to leave out basic support packages like os or glob.)_

**Credit:**
_E.g., "Originally developed by" or "Based on notebooks developed by" and then people's names, including journal article or software release citations if appropriate._
Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

_Provide a light narrative about this notebook, e.g., "This notebook will teach the user..."._

_Cite or link to any external information or documentation, and cross-reference to other notebooks._

### 1.1 Package Imports

_All package imports should be done in the first code cell._

_Provide explanation or external links to package documentation, where appropriate._

_E.g., Numpy is a fundamental package for scientific computing with arrays in Python (<a href="https://numpy.org">numpy.org</a>)._

_Use code cell comments to describe the packages being imported._

In [8]:
# general python packages
import time
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
import pandas
pandas.set_option('display.max_rows', 1000)

# LSST package for TAP queries
from lsst.rsp import get_tap_service, retrieve_query

# LSST package for Butler queries
import lsst.daf.butler as Butler

# LSST package for image display
import lsst.afw.display as afwDisplay
import lsst.geom


### 1.2 Define Functions and Parameters

_If your notebook defines functions or parameters to use later or throughout, do it here in sub-section 1.2._

_It is OK to rename the subsection to be more specific to the notebook, and/or to use sub-sub-sections like "1.2.1 Define global cosmological parameter values" or "1.2.2 Define a function to make an image cutout"._

_It is OK to remove this sub-section if it is not being used._

## 2. Search data for a supernova for which to create image cutouts

_Use numbers for sections, sub-sections, and sub-sub-sections to enable referencing, e.g., "I'm having trouble with the second code cell in Section 2.3."_

_Use section titles that actively describe what is being done, e.g., "Create a color-magnitude diagram" instead of "Plot", so that the auto-generated table of contents is easy to navigate._

### 2.1 Section Sub-heading

#### 2.1.1 Section Sub-sub-heading

In [3]:
service = get_tap_service()
assert service is not None

In [4]:
# galaxy cluster coords: 55.6521739130, -31.9834710744
# grab a DiaObject identified in Tutorial 07a to be a candidate Type 1a SN with relatively few number of 
# nDiaSources (frames over which flux varies): RA=51.2332445 Dec=-44.509536 DiaObjID=1248649384967537762
#%%time

# First setup the search like in tutorial 07a:
redshift_min = 0.1
redshift_max = 0.3
snia_peak_mag = -19.0
snia_peak_mag_range = 0.5
snia_peak_mg_max = 24.0
snia_peak_mi_max = 24.0
snia_ampl_mr_min = 1.5
snia_ampl_mr_max = 5.5
snia_peak_mr_min = 18.82
snia_peak_mr_max = 22.46
# pick a set on the smaller end of time to make the output cutouts managable:
snia_nDiaSources_min = 15
snia_nDiaSources_max = 20


   

In [6]:
%%time

results = service.search("SELECT ra, decl, diaObjectId, nDiaSources, "
                         "scisql_nanojanskyToAbMag(rPSFluxMin) AS rMagMax, "
                         "scisql_nanojanskyToAbMag(rPSFluxMax) AS rMagMin, "
                         "scisql_nanojanskyToAbMag(gPSFluxMax) AS gMagMin, "
                         "scisql_nanojanskyToAbMag(iPSFluxMax) AS iMagMin, "
                         "scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) AS rMagAmp "
                         "FROM dp02_dc2_catalogs.DiaObject "
                         "WHERE nDiaSources > "+str(snia_nDiaSources_min)+" "
                         "AND nDiaSources < "+str(snia_nDiaSources_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMax) > "+str(snia_peak_mr_min)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMax) < "+str(snia_peak_mr_max)+" "
                         "AND scisql_nanojanskyToAbMag(gPSFluxMax) < "+str(snia_peak_mg_max)+" "
                         "AND scisql_nanojanskyToAbMag(iPSFluxMax) < "+str(snia_peak_mi_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) < "+str(snia_ampl_mr_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) > "+str(snia_ampl_mr_min)+" ",
                         maxrec=1000)

DiaObjs = results.to_table()
del results

CPU times: user 52 ms, sys: 2.16 ms, total: 54.1 ms
Wall time: 6.58 s


In [4]:
#display the table
DiaObjs

ra,decl,diaObjectId,nDiaSources,rMagMax,rMagMin,gMagMin,iMagMin,rMagAmp
deg,deg,,,,,,,
float64,float64,int64,int64,float64,float64,float64,float64,float64
51.2332445,-44.509536,1248649384967537762,19,23.735368714085325,18.97590373930018,18.198346264103098,19.814439457113927,4.759464974785146
51.6735058,-44.4932479,1248631792781492328,19,23.58561832547451,19.59055504249835,20.143623389883878,22.03210883958532,3.995063282976158
59.6711869,-44.5760311,1250364623106867975,17,21.19770251002847,19.55007375592156,18.63866129605477,17.218179024838108,1.6476287541069112
59.2766834,-44.5136323,1250373419199889677,18,23.38321000558014,21.69423956763242,21.259810263411886,22.32197599532804,1.6889704379477202
68.8790005,-44.5228684,1252546054176374863,17,24.942905167752485,20.440392039021507,17.970678999672423,20.99523478783776,4.502513128730978
60.368959,-44.582947,1250830816037045527,19,23.532239879577165,21.099157003887804,21.811210109072316,21.60654191540347,2.4330828756893617
55.9495772,-44.549466,1249977595013890139,18,26.580919769065176,22.375728744981714,23.05708741170058,23.307145527229494,4.205191024083462
55.9495772,-44.549466,1249493809897668676,18,26.580919769065176,22.375728744981714,23.05708741170058,23.307145527229494,4.205191024083462


In [9]:
# Initialize the Butler  
config = 'dp02'
collection = '2.2i/runs/DP0.2'
butler = Butler.Butler(config, collections=collection)
registry = butler.registry

In [10]:
ra, dec = DiaObjs['ra'], DiaObjs['decl']
#cutout = cutout_coadd(butler, ra, dec, band='g', datasetType='deepCoadd', cutoutSideLength=1000)

# To get the visits used to contruct the deepCoadd
#unique_visits = list(set([i['visit'] for i in cutout.getInfo().getCoaddInputs().ccds]))

# Look up the tract, patch for the RA, Dec
tractInfo = skymap.findTract(radec)
patchInfo = tractInfo.findPatch(radec)
xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
patch = tractInfo.getSequentialPatchIndex(patchInfo)

coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
parameters = {'bbox': bbox}

cutout_image = butler.get(datasetType, parameters=parameters,
                           dataId=coaddId)

NameError: name 'cutout_coadd' is not defined

In [11]:
Adapted from DC2 tutorial notebook by Michael Wood-Vasey.

Parameters
----------
butler: lsst.daf.persistence.Butler
    Servant providing access to a data repository
ra: float
    Right ascension of the center of the cutout, in degrees
dec: float
    Declination of the center of the cutout, in degrees
band: string
    Filter of the image to load
datasetType: string ['deepCoadd']
    Which type of coadd to load.  Doesn't support 'calexp'
skymap: lsst.afw.skyMap.SkyMap [optional]
    Pass in to avoid the Butler read.  Useful if you have lots of them.
cutoutSideLength: float [optional]
    Size of the cutout region in pixels.

Returns
-------
MaskedImage
"""
radec = geom.SpherePoint(ra, dec, geom.degrees)
cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

if skymap is None:
    skymap = butler.get("skyMap")

# Look up the tract, patch for the RA, Dec
tractInfo = skymap.findTract(radec)
patchInfo = tractInfo.findPatch(radec)
xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
patch = tractInfo.getSequentialPatchIndex(patchInfo)

coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
parameters = {'bbox': bbox}

cutout_image = butler.get(datasetType, parameters=parameters,
                           dataId=coaddId)

return cutout_image

SyntaxError: unterminated string literal (detected at line 14) (2384229232.py, line 14)